In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Reshape
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

In [2]:
no_of_patients=75

In [3]:
train=[]
test=[]
for i in range(1,no_of_patients+1):
    files=glob('C:/Users/kotha/Desktop/matplot/eeg/files/S'+str(str(i).zfill(3))+'/*.edf')
    files=files[4:5]
    train+=files
train

['C:/Users/kotha/Desktop/matplot/eeg/files/S001\\S001R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S002\\S002R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S003\\S003R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S004\\S004R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S005\\S005R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S006\\S006R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S007\\S007R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S008\\S008R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S009\\S009R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S010\\S010R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S011\\S011R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S012\\S012R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S013\\S013R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S014\\S014R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S015\\S015R05.edf',
 'C:/Users/kotha/Desktop/matplot/eeg/files/S016\\S016R0

In [4]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
#     display(eeg_df)
    eeg_df = eeg_df.iloc[:15000]
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

In [5]:
def scale_dataset(dataframe):
    x=dataframe[dataframe.columns[:-1]].values
    y=dataframe[dataframe.columns[-1]].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)  
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid
display(xtrain)
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [7]:
tf.random.set_seed(1234)
model = Sequential()
#model.add(Reshape((64,1),input_shape=(64,)))
model.add(Reshape((64,1)))
model.add(Conv1D(32, kernel_size=11, activation='relu',input_shape=(75,12000,64)))
model.add(Conv1D(64, kernel_size=11, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=11, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(75, activation='softmax'))

In [8]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
28125/28125 [==============================] - 264s 9ms/step - loss: 0.7142 - val_loss: 1.2868
Epoch 2/10
28125/28125 [==============================] - 258s 9ms/step - loss: 0.3493 - val_loss: 1.1827
Epoch 3/10
28125/28125 [==============================] - 260s 9ms/step - loss: 0.2943 - val_loss: 1.3044
Epoch 4/10
28125/28125 [==============================] - 274s 10ms/step - loss: 0.2698 - val_loss: 1.3464
Epoch 5/10
28125/28125 [==============================] - 269s 10ms/step - loss: 0.2549 - val_loss: 1.2666
Epoch 6/10
28125/28125 [==============================] - 249s 9ms/step - loss: 0.2461 - val_loss: 1.2471
Epoch 7/10
28125/28125 [==============================] - 290s 10ms/step - loss: 0.2384 - val_loss: 1.3894
Epoch 8/10
28125/28125 [==============================] - 266s 9ms/step - loss: 0.2321 - val_loss: 1.3584
Epoch 9/10
28125/28125 [==============================] - 274s 10ms/step - loss: 0.2301 - val_loss: 1.3691
Epoch 10/10
28125/28125 [=================

In [9]:
y_pred=model.predict(x)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y,y_pred_classes))

28125/28125 [==============================] - 107s 4ms/step
              precision    recall  f1-score   support

           0       0.96      0.85      0.90     12000
           1       0.97      0.92      0.94     12000
           2       0.94      0.95      0.95     12000
           3       0.98      0.98      0.98     12000
           4       0.99      0.98      0.99     12000
           5       0.99      0.98      0.99     12000
           6       0.95      0.97      0.96     12000
           7       0.97      0.99      0.98     12000
           8       0.97      0.95      0.96     12000
           9       0.98      0.97      0.98     12000
          10       0.97      0.99      0.98     12000
          11       0.99      0.99      0.99     12000
          12       0.95      0.83      0.88     12000
          13       0.95      0.94      0.95     12000
          14       0.87      0.82      0.84     12000
          15       0.97      0.99      0.98     12000
          16       0

In [10]:
y_pred=model.predict(xv)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(yv,y_pred_classes))

3514/3514 [==============================] - 16s 4ms/step
              precision    recall  f1-score   support

           0       0.39      0.16      0.23      1499
           1       0.85      0.41      0.55      1499
           2       0.79      0.91      0.85      1499
           3       0.90      0.86      0.88      1499
           4       0.95      0.95      0.95      1499
           5       0.97      0.91      0.94      1499
           6       0.88      0.94      0.91      1499
           7       0.93      0.98      0.95      1499
           8       0.74      0.87      0.80      1499
           9       0.77      0.76      0.76      1499
          10       0.86      0.92      0.89      1499
          11       0.96      0.97      0.96      1499
          12       0.85      0.82      0.83      1499
          13       0.86      0.89      0.88      1499
          14       0.53      0.68      0.59      1499
          15       0.90      0.90      0.90      1499
          16       0.90

In [11]:
y_pred=model.predict(xt)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(yt,y_pred_classes))

3514/3514 [==============================] - 16s 4ms/step
              precision    recall  f1-score   support

           0       0.82      0.66      0.73      1499
           1       0.91      0.61      0.73      1499
           2       0.83      0.84      0.83      1499
           3       0.92      0.86      0.89      1499
           4       0.95      0.98      0.96      1499
           5       0.98      0.90      0.94      1499
           6       0.88      0.97      0.92      1499
           7       0.94      0.96      0.95      1499
           8       0.80      0.88      0.84      1499
           9       0.81      0.73      0.77      1499
          10       0.83      0.99      0.90      1499
          11       0.97      0.98      0.97      1499
          12       0.87      0.83      0.85      1499
          13       0.86      0.86      0.86      1499
          14       0.69      0.68      0.69      1499
          15       0.90      0.95      0.92      1499
          16       0.91